# Using Keypoint by mediapipe

In [1]:
import os
import random
import pandas as pd
import numpy as np

import cv2
import mediapipe as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Hyperparameter Setting

In [2]:
CFG = {
    'FPS':30,
    'IMG_SIZE':128,
    'EPOCHS':30,
    'LEARNING_RATE':2e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### Data Load

In [4]:
data_root_path = r"D:\AI_Data\TV_HMCR"

In [5]:
df_train = pd.read_csv(os.path.join(data_root_path, "train.csv"))
df_train["path"] = df_train["path"].apply(lambda x: os.path.join(data_root_path, os.path.join(x.split("/")[1], x.split("/")[2])))
display(df_train.head(3))

df_test = pd.read_csv(os.path.join(data_root_path, "test.csv"))
df_test["path"] = df_test["path"].apply(lambda x: os.path.join(data_root_path, os.path.join(x.split("/")[1], x.split("/")[2])))
display(df_test.head(3))

df_train.shape, df_test.shape

,id,path,label
0,TRAIN_000,D:\AI_Data\TV_HMCR\train\TRAIN_000.mp4,3
1,TRAIN_001,D:\AI_Data\TV_HMCR\train\TRAIN_001.mp4,0
2,TRAIN_002,D:\AI_Data\TV_HMCR\train\TRAIN_002.mp4,1


,id,path
0,TEST_000,D:\AI_Data\TV_HMCR\test\TEST_000.mp4
1,TEST_001,D:\AI_Data\TV_HMCR\test\TEST_001.mp4
2,TEST_002,D:\AI_Data\TV_HMCR\test\TEST_002.mp4


((610, 3), (153, 2))

### Resize

In [6]:
def get_video(path):
    frames = []
    cap = cv2.VideoCapture(path)
    for _ in range(CFG["FPS"]):
        _, img = cap.read()
        img = cv2.resize(img, (CFG["IMG_SIZE"], CFG["IMG_SIZE"]))
        img = img / 255.
        frames.append(img)
    return frames

### Train / Validation Split

In [7]:
train, val, _, _ = train_test_split(df_train, df_train["label"], test_size=0.2, random_state=CFG["SEED"])

train.shape, val.shape

((488, 3), (122, 3))

### Keypoint Extract

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [9]:
from copy import deepcopy
from sklearn.metrics.pairwise import itertools

class Detecter():
    def __init__(self, mode=False, maxHands=2, detection_conf=1, track_conf=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detection_conf = detection_conf
        self.track_conf = track_conf
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detection_conf, self.track_conf)
        self.mpDraw = mp.solutions.drawing_utils
        
    def find_hands(self, img, draw=False):
        rgb_img = cv2.flip(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), 1)
        self.results = self.hands.process(rgb_img)
        if self.results.multi_hand_landmarks:
            for hand_mk, hand_ness in zip(self.results.multi_hand_landmarks, self.results.multi_handedness):
                if draw: self.mpDraw.draw_landmarks(img, hand_mk, self.mpHands.HAND_CONNECTIONS)
        return img
    
    def find_position(self, img, hand_no=0, draw=False):
        mk = []
        if self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[hand_no]
            for _, mk_part in enumerate(hand.landmark):
                h, w, c = img.shape
                cx, cy = min(int(mk_part.x*w), w-1), min(int(mk_part.y*h), h-1)
                mk.append([cx, cy])
                if draw: cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        return mk
    
    def preprocess_lmk(self, mk):
        temp_lmk_list = deepcopy(mk)
        # conver coor
        base_x, base_y = 0, 0
        for id, lmk in enumerate(temp_lmk_list):
            if id==0: base_x, base_y = mk[0], mk[1]
            temp_lmk_list[id][0] = temp_lmk_list[id][0] - base_x
            temp_lmk_list[id][1] = temp_lmk_list[id][1] - base_y
        # convert 1d
        temp_lmk_list = list(itertools.chain.from_iterable(temp_lmk_list)) 
        # Normalize
        max_value = max(list(map(abs, temp_lmk_list)), default=0)
        return temp_lmk_list

In [ ]:
detecter = Detecter()

train_lmk = []

for v in range(df_train.shape[0]):
    vPath = df_train.iloc[v]["path"]
    
    cap = cv2.VideoCapture(vPath)
    while True:
        ret, img = cap.read()
        if not ret: break
        img = detecter.find_hands(img)
        lmk = detecter.find_position(img)
        
        train_lmk.append(lmk)
        print(v, len(lmk))

In [39]:
v_dict = {}
train_vpath_key = [f"v{id}" for id in range(df_train.shape[0])]
lmk_range = [idx for idx in range(0, len(train_lmk), 30)]

for k, v in zip(train_vpath_key, lmk_range):
    v_dict[k] = train_lmk[v:v+CFG["FPS"]]

In [64]:
cor = pd.DataFrame(v_dict)
cor = cor.apply(lambda x: np.array(x))

In [71]:
cor["v0"]

0     [[23, 88], [23, 79], [19, 67], [16, 58], [13, ...
1     [[23, 90], [24, 79], [22, 68], [19, 60], [17, ...
2     [[24, 91], [23, 78], [20, 67], [17, 61], [15, ...
3     [[25, 92], [24, 79], [22, 68], [18, 61], [14, ...
4     [[26, 92], [24, 79], [22, 66], [18, 59], [15, ...
5     [[26, 93], [24, 80], [22, 68], [19, 60], [15, ...
6     [[28, 94], [28, 80], [25, 68], [21, 61], [18, ...
7     [[29, 94], [28, 80], [26, 67], [22, 60], [19, ...
8     [[31, 95], [30, 81], [28, 69], [24, 62], [21, ...
9     [[34, 96], [33, 83], [30, 71], [27, 63], [24, ...
10    [[36, 96], [36, 84], [33, 72], [30, 64], [28, ...
11    [[38, 97], [33, 90], [30, 80], [29, 72], [26, ...
12    [[42, 97], [39, 87], [36, 75], [34, 67], [32, ...
13    [[44, 98], [40, 88], [37, 75], [36, 66], [36, ...
14    [[48, 98], [42, 86], [38, 72], [37, 61], [37, ...
15    [[52, 97], [48, 90], [45, 79], [43, 69], [42, ...
16    [[55, 96], [51, 92], [48, 82], [46, 75], [44, ...
17    [[58, 95], [52, 90], [49, 78], [48, 68], [

In [58]:
type(cor.iloc[0, 0])

list